In [ ]:
from utils import *
from HeteTrans import *

hete = Heterogeneity()
hete.check_torch_gpu()

***
# 1

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from cv2 import resize
from scipy.io import loadmat
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from vformer.models.classification.vivit import ViViTModel2, ViViTEncoder
from vformer.encoder.embedding.video_patch_embeddings import TubeletEmbedding, LinearVideoEmbedding

/home/misaelmorales/miniconda3/envs/torchy/lib/python3.10/site-packages/vformer-0.1.3-py3.10.egg/vformer/models/classification/cvt.py:121: SyntaxWarning: "is" with a literal. Did you mean "=="?
/home/misaelmorales/miniconda3/envs/torchy/lib/python3.10/site-packages/vformer-0.1.3-py3.10.egg/vformer/models/classification/cvt.py:121: SyntaxWarning: "is" with a literal. Did you mean "=="?


In [5]:
print(xbatch.shape)
print(ybatch.shape)

torch.Size([50, 30, 3, 64, 64])
torch.Size([50, 30, 2, 64, 64])


In [31]:
model = TubeletEmbedding(embedding_dim=64, tubelet_t=3, tubelet_h=4, tubelet_w=3, in_channels=3)

z = model(torch.moveaxis(xbatch, 2, -1))
print(z.shape)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (8000x2304 and 108x64)

In [ ]:
w = nn.ConvTranspose3d(5, 15, 3, padding=0, stride=2)(z)
print(w.shape)
w = nn.Conv3d(15, 15, (35,3,3), padding=1)(w)
print(w.shape)

print('-----------')

w = nn.ConvTranspose3d(15, 30, 3, padding=0, stride=2)(w)
print(w.shape)
w = nn.Conv3d(30, 30, (19,3,3), padding=1)(w)
print(w.shape)

In [2]:
class NormalizeTransform:
    def __call__(self, sample):
        x, y = sample
        x_normalized = self.normalize_data(x)
        y_normalized = self.normalize_data(y)
        return x_normalized, y_normalized

    def normalize_data(self, data):
        scaler = MinMaxScaler()
        data_np = data.numpy()
        data_normalized_np = scaler.fit_transform(data_np.reshape(-1, data_np.shape[-1])).reshape(data_np.shape)
        return torch.Tensor(data_normalized_np)

In [3]:
class MyDataset(Dataset):
    def __init__(self, data_folder, start, end, transform=None):
        self.y_folder = data_folder
        self.x_folder = os.path.join(data_folder, 'X_data')
        self.y_list = [file for file in os.listdir(self.y_folder) if file.endswith('.mat')]
        self.x_list = os.listdir(self.x_folder)
        self.transform = transform
        self.start, self.end = start, end

    def __len__(self):
        return len(self.x_list)

    def __getitem__(self, idx):
        x_name = self.x_list[idx]
        y_name = self.y_list[idx]

        x_path = os.path.join(self.x_folder, x_name)
        y_path = os.path.join(self.y_folder, y_name)

        x_data = np.load(x_path)
        poro = torch.tensor(resize(x_data[0], (64, 64)), dtype=torch.float32).unsqueeze(0).repeat(1, 61, 1, 1)
        perm = torch.tensor(resize(np.log10(x_data[1]), (64, 64)), dtype=torch.float32).unsqueeze(0).repeat(1, 61, 1, 1)
        time = torch.tensor(np.arange(61).reshape(1, 61, 1, 1), dtype=torch.float32).repeat(1, 1, 64, 64)
        x = torch.cat([poro, perm, time], dim=0).permute(1,0,2,3)

        y = torch.zeros((2, 61, 64, 64))
        y_data = loadmat(y_path, simplify_cells=True)
        for timestep in range(61):
            y[0, timestep] = torch.tensor(resize(y_data['PRESSURE'], (64, 64)), dtype=torch.float32)
            y[1, timestep] = torch.tensor(resize(y_data['SGAS'], (64, 64)) * resize(y_data['YMF_3'], (64, 64)), dtype=torch.float32)
        y = y.permute(1,0,2,3)

        sample = (x[self.start:self.end], y[self.start:self.end])

        if self.transform:
            sample = self.transform(sample)

        return sample

In [4]:
transform  = NormalizeTransform()
traindataset = MyDataset(data_folder='h2dataf', start=0, end=30)
validdataset = MyDataset(data_folder='h2dataf', start=30, end=45)
testdataset  = MyDataset(data_folder='h2dataf', start=45, end=-1)

train_loader = DataLoader(traindataset, batch_size=50, shuffle=True)
valid_loader = DataLoader(validdataset, batch_size=50, shuffle=True)
test_loader  = DataLoader(testdataset, batch_size=50, shuffle=True)

for i, (xbatch, ybatch) in enumerate(train_loader):
    if i<2:
        print(xbatch.shape, ybatch.shape)

torch.Size([50, 30, 3, 64, 64]) torch.Size([50, 30, 2, 64, 64])
torch.Size([50, 30, 3, 64, 64]) torch.Size([50, 30, 2, 64, 64])


KeyboardInterrupt: 

In [ ]:
from transformers import VivitModel, VivitConfig

In [ ]:
vivit_config = VivitConfig(image_size=64, num_frames=30, hidden_size=1024, num_attention_heads=8)

In [ ]:
xbatch.shape

In [ ]:
z = VivitModel(vivit_config)(xbatch)

In [ ]:
z.pooler_output.shape

In [ ]:
from vformer.encoder import SwinEncoder

In [ ]:
model      = ViTConvModel()
criterion  = nn.MSELoss()
optimizer  = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs   = 10
train_tsteps = 40 
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    average_loss = total_loss / len(train_loader)
    print('Epoch: [{}/{}] | Batch: [{}/{}] | Loss: {}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

model.eval()
with torch.no_grad():
    total_val_loss = 0.0
    for x_val, y_val in valid_loader:
        val_outputs = model(x_val)
        val_loss = criterion(val_outputs, y_val)
        total_val_loss += val_loss.item()
    average_val_loss = total_val_loss / len(valid_loader)
    print(f"Validation Loss: {average_val_loss}")

***

In [ ]:
### OLD ###

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels, patch_size, embed_dim):
        super(PatchEmbedding, self).__init__()
        self.patch_size = patch_size
        self.projection = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.batchnorm  = nn.BatchNorm2d(embed_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.projection(x)
        x = self.batchnorm(x)
        x = self.activation(x)
        return x
    
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads)

    def forward(self, x):
        return self.attention(x, x, x)[0]
    
class VisionTransformer(nn.Module):
    def __init__(self, in_channels, patch_size, embed_dim, num_heads, num_layers):
        super(VisionTransformer, self).__init__()
        self.patch_embedding    = PatchEmbedding(in_channels, patch_size, embed_dim)
        self.transformer_layers = nn.ModuleList([ nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads) for _ in range(num_layers) ])
        self.fc                 = nn.Linear(embed_dim, in_channels)
        self.batchnorm          = nn.BatchNorm1d(in_channels)

    def forward(self, x):
        x = self.patch_embedding(x)
        x = x.flatten(2).permute(2, 0, 1)
        for layer in self.transformer_layers:
            x = layer(x)
        x = x.permute(1, 2, 0).view(x.size(1), -1, x.size(0))
        x = self.fc(x)
        x = self.batchnorm(x)
        x = F.gelu(x)
        return x